## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [5]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
import warnings
warnings.filterwarnings('ignore')

In [6]:
wine = datasets.load_wine()
x_train, x_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.25, random_state=42)
clf = GradientBoostingRegressor(random_state=7)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(f'MSE:{metrics.mean_squared_error(y_test, y_pred)}')

MSE:0.14672894294872418


In [7]:
# 設定要訓練的超參數組合
n_estimators = [x for x in range(20, 300, 10)]
max_depth = [x for x in range(1, 15, 2)]
param_range = dict(n_estimators=n_estimators, max_depth=max_depth)

# 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
random_search = RandomizedSearchCV(clf, param_range, scoring="neg_mean_squared_error", 
                                   random_state=1, n_iter=100, cv=5, verbose=0, n_jobs=-1)

# 開始搜尋最佳參數
random_result = random_search.fit(x_train, y_train)
print("Best Accuracy: %f using %s" % (random_result.best_score_, random_result.best_params_))

Best Accuracy: -0.065170 using {'n_estimators': 30, 'max_depth': 9}


In [8]:
random_result.best_params_

{'n_estimators': 30, 'max_depth': 9}

In [12]:
clf_bestparam = GradientBoostingRegressor(max_depth=random_result.best_params_['max_depth'],
                                           n_estimators=random_result.best_params_['n_estimators'])
clf_bestparam.fit(x_train, y_train)
y_pred = clf_bestparam.predict(x_test)
print(f'MSE:{metrics.mean_squared_error(y_test, y_pred)}')

MSE:0.17472835580260596
